In [ ]:
%pip install recviz
%pip install graphviz
# !pip install binarytree
# !pip install recursion-visualiser


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#remove duplicate rows
def remove_duplicate(np_f,np_g):
    unique_f = np.unique(np_f,axis=0)
    unique_g = np.unique(np_g,axis=0)
    return unique_f, unique_g

In [ ]:
#remove super set
def remove_superset(np_arr):
    #sort the array by row sum(max to min)
    np_arr = sorted(np_arr, key=lambda x: sum(x),reverse=True)
    np_arr = np.asarray(np_arr)
    # store the row indexes to be deleted
    delete_rows = []
    for i in range(len(np_arr)):
        # find the column where the value is 0
        zero_index = np.where(np_arr[i] == 0)[0]
        if(len(zero_index) != 0):
            zero_cols = np_arr[:, np.array(zero_index)]
            rows_sum = np.sum(zero_cols, axis=1)
            # check if there exists another row where the value are all-zeros at these columns
            if np.count_nonzero(rows_sum == 0) > 1:
                delete_rows.append(i)
    delete = np_arr
    np_arr = np.delete(np_arr,delete_rows,0)
#    return np_arr
    return np_arr, delete[delete_rows]



In [ ]:
# remove_superset test sample:
np_arr = np.array([[1,0,1,0,0,0,0,0],
                [1,0,0,1,0,0,0,0],
                [0,1,1,0,0,0,0,0],
                [0,1,1,0,1,0,1,0],
                [0,1,0,1,0,0,0,0],
                [0,0,0,0,1,0,1,0],
                [1,0,0,1,0,1,1,0],
                [0,0,0,0,1,0,0,1],
                [0,0,0,0,0,1,1,0],
                [0,0,0,0,0,1,0,1]])
print("Sperner Hypergraph, Deleted Edges: ", remove_superset(np_arr))

In [ ]:
# display the data frame with column total and row total attributes
def display(f,g):
    f=pd.DataFrame(unique_f)
    f.loc['Column_Total']= f.sum(numeric_only=True, axis=0)
    fmax_col_index = np.argmax(np.array(f.loc['Column_Total']))
    f.loc[:,'Row_Total'] = f.sum(numeric_only=True,axis=1)
    fmax_row_value=max(f['Row_Total'][0:-1])
    df_f=f
    print(df_f)
    print("the variable that appears max in f is: X",fmax_col_index)

    g=pd.DataFrame(unique_g)
    g.loc['Column_Total']= g.sum(numeric_only=True, axis=0)
    gmax_col_index = np.argmax(np.array(g.loc['Column_Total']))
    g.loc[:,'Row_Total'] = g.sum(numeric_only=True,axis=1)
    gmax_row_value=max(g['Row_Total'][0:-1])
    df_g=g
    print(df_g)
    print("the variable that appears max in g is：X",gmax_col_index)
    return df_f,df_g

In [ ]:
# pre-req
#np_f and np_g are numpy arrays
def check_pre(f,g):
# input: f, g -- numpy array
# 1. assume var in f and g are the same, and we check if # of columns in f == # of columns in g
    if np.shape(f)[1] != np.shape(g)[1]:
        print("not dual, # of columns in f != # of columns in g")
        return False
# 2.check if largest of sum_row in f <= # of rows in g, largest value of sum_row in g <= # of rows in f
    fmax_sum_row = max(f.sum(axis=1))
    gmax_sum_row = max(g.sum(axis=1))
    if fmax_sum_row > np.shape(g[0]):
        print("not dual, fmax_row_value > len(g)")
        return False

    if gmax_sum_row > np.shape(f[0]):
        print("not dual, gmax_row_value > len(f)")
        return False
# 3. sum 2^(-|c in f|)+sum 2^(-|c in g|) >=1, where c is each sum_row value(for loop)>=1
    sum_fg = 0
    f_row_sum_array = f.sum(axis=1)
    g_row_sum_array = g.sum(axis=1)

    for i in range(np.shape(f)[0]):
        sum_fg += 1/(2**(f_row_sum_array[i]))
    for i in range(np.shape(g)[0]):
        sum_fg += 1/(2**(g_row_sum_array[i]))
    if sum_fg < 1:
        print("not dual, sum_fg < 1")
        return False
#Condition 4 means when we compare any two rows of the input matrices, they should have at least one
#non-empty intersection
# 4. C ^ C' != empty
    # need to check if the one is in the same position
    #         import pdb;  pdb.set_trace()
    for i in range(len(f)):
        for j in range(np.shape(f)[1]):
            if f[i][j] ^ g[i][j] == True:
                pass
            else:
                return True
        return False


In [ ]:
#Storing variables for each node
f_remover = []
g_remover = []
f_deleted_rows = []
g_deleted_rows = []
f0_dict = {}
f1_dict = {}
g0_dict = {}
g1_dict = {}
spliting_dict = {}
x = None

In [ ]:
import sys
from functools import wraps
from collections import OrderedDict
import pydot
import imageio
import glob
import os
import shutil

# Dot Language for graph
dot_str_start = "digraph G {\n"
dot_str_body = ""
dot_str_end = "}"


class Visualiser(object):
    def __init__(self, ignore_args=None, show_argument_name=True,
                 show_return_value=True, node_properties_kwargs={}):
        self.init_graph()
        # If enabled shows keyword arguments ordered by keys
        self.show_argument_name = show_argument_name
        # If enables shows the return value at every nodes
        self.show_return_value = show_return_value

        self.node_properties_kwargs = node_properties_kwargs

        # Argument string that are to be ignored in diagram
        if ignore_args is not None:
            self.ignore_args = ignore_args


    @classmethod
    def write_image(cls, filename="out.png"):
        try:
            cls.graph.write_png(f"{filename}")
            print(f"File {filename} successfully written")
        except Exception:
            print(f"Writing {filename} failed")

    @classmethod
    def make_frames(cls):
        """
        Make frame for each steps
        """
        # If frame directory doesn't exist
        if not os.path.exists("frames"):
            os.makedirs("frames")

        Edges = cls.edges[::]
        Nodes = cls.nodes[::]
        #print("Writing frames....")
        for i in range(len(Edges)):
            nodes = Nodes[::]
            edges = Edges[::]

            for j in range(0, i + 1):
                nodes[j] += '];'

            for j in range(i + 1, len(Edges)):
                nodes[j] += ' , style=invis];'
                edges[j] += ' [style=invis];'

            dot_str_body = "\n".join(nodes) + "\n"
            dot_str_body += "\n".join(edges)
            dot_str = dot_str_start + dot_str_body + dot_str_end
            g = pydot.graph_from_dot_data(dot_str)
            g[0].write_png(f"frames/temp_{i}.png")

    @classmethod
    def write_gif(cls, name="out.gif", delay=3):
        images = []

        # sort frames images in ascending order to number in image filename
        # image filename: frames/temp_1.png
        sorted_images = sorted(
            glob.glob("frames/*.png"),
            key=lambda fn: int(fn.split("_")[1].split(".")[0])
        )

        for filename in sorted_images:
            images.append(imageio.imread(filename))
        #print("Writing gif...")
        imageio.mimsave(name, images, duration=delay)
        #print(f"Saved gif {name} successfully")
        # Delete temporary directory
        shutil.rmtree("frames")

    @classmethod
    def make_animation(cls, filename="out.gif", delay=3):
        # print("Starting to make animation")
        # Save final tree image as png
        try:
            cls.write_image(f"{filename.split('.')[0]}.png")
        except:
            print("Error saving image.")

        # Make animation as gif
        try:
            cls.make_frames()
        except:
            print("Error writing frames")

        try:
            cls.write_gif(filename, delay=delay)
        except:
            print("Error saving gif.")

        cls.init_graph()

    def extract_arg_strings(self, *args, **kwargs):
        """
        Returns function signature arguments function label arguments as
        string.
        label_args_string contains only the arguments that are not in
        ignore_args.
        signature_args_string contains all the arguments available for the
        function.
        """

        def get_kwargs_strings(ignore_args=[]):
            """Returns list of kwargs in string format from given kwargs items

            Args:
                ignore_args (list, optional) : list of ignored arguments.
                Default to [].

            Returns:
                strings_list: list of kwargs in string format
            """

            strings_list = []

            for key, value in kwargs.items():
                if key not in ignore_args:
                    if not self.show_argument_name:
                        strings_list.append(f"\n{repr(value)}")
                    else:
                        strings_list.append(f"\n{key}={repr(value)}")

            strings_list = strings_list[-1:] + strings_list[:-1]
            # if x in spliting_dict:
            #   key = "Frequent Variable"
            #   strings_list.append(f"{key}={spliting_dict[x]}")
            return strings_list

        args_string = [repr(a) for a in args]
        signature_kwargs_string = [f"{repr(kwargs.get('node_num'))}"]
        label_kwargs_string = get_kwargs_strings(ignore_args=self.ignore_args)

        signature_args_string = ', '.join(signature_kwargs_string)
        label_args_string = ', '.join(args_string + label_kwargs_string)

        return signature_args_string, label_args_string

    def string2int(self, string):
      num = ""
      for c in string:
        if c.isdigit():
          num = num + c
      return int(num)

    def __call__(self, fn):
        @ wraps(fn)
        def wrapper(*args, **kwargs):
            global dot_str_body
            # Increment total number of nodes when a call is made
            self.node_count += 1

            # Update kwargs by adding dummy keyword node_num which helps to
            # uniquely identify each node
            kwargs.update({'node_num': self.node_count})
            # Order all the keyword arguments
            kwargs = OrderedDict(sorted(kwargs.items()))

            """Details about current Function"""
            # Get signature and label arguments strings for current function
            (signature_args_string,
             label_args_string) = self.extract_arg_strings(
                 *args, **kwargs)

            # Details about current function
            function_name = fn.__name__

            # Current function signature looks as follows:
            # foo(1, 31, 0) or foo(a=1, b=31, c=0)
            # function_signature = f"{function_name}({signature_args_string})"
            # function_label = f"{function_name}({label_args_string})"
            function_signature = f"{function_name}({signature_args_string})"
            function_label = f"{function_name}({label_args_string})"
            """"""

            """Details about caller function"""
            caller_func_frame = sys._getframe(1)
            # All the argument names in caller/parent function
            caller_func_arg_names = caller_func_frame.f_code.co_varnames[
                : fn.__code__.co_argcount]
            caller_func_locals = caller_func_frame.f_locals
            # Sort all the locals of caller function
            caller_func_locals = OrderedDict(
                sorted(caller_func_locals.items()))

            caller_func_kwargs = dict()

            # Extract only those locals that are in arguments
            for key, value in caller_func_locals.items():
                if key in caller_func_arg_names:
                    caller_func_kwargs[key] = value

            # If the nodes has parent node get node_num from parent node
            if self.stack:
                caller_func_kwargs.update({'node_num': self.stack[-1]})

            caller_func_kwargs = OrderedDict(
                sorted(caller_func_kwargs.items()))

            (caller_func_args_string,
             caller_func_label_args_string) = self.extract_arg_strings(
                **caller_func_kwargs)

            # Caller Function
            caller_func_name = caller_func_frame.f_code.co_name

            # Extract the names of arguments only
            caller_func_signature = "{}({})".format(
                caller_func_name, caller_func_args_string)
            caller_func_label = "{}({})".format(
                caller_func_name, caller_func_label_args_string)
            """"""

            if caller_func_name == '<module>':
                print(f"Drawing for {function_signature}")

            # Push node_count to stack
            self.stack.append(self.node_count)
            # Before actual function call delete keyword 'node_num' from kwargs
            print(f"-------------------")
            global x
            x = kwargs['node_num']
            print(f"Node Number: {kwargs['node_num']}")
            print(f"-------------------")

            del kwargs['node_num']

            self.edges.append(
                f'"{caller_func_signature}" -> "{function_signature}"')

            # Construct node string to be rendered in graphviz
            node_string = f'"{function_signature}" [label="{function_label}"'

            if self.node_properties_kwargs:
                node_string += ", " + \
                    ", ".join([f'{key}="{value}"' for key,
                               value in self.node_properties_kwargs.items()])

            self.nodes.append(node_string)

            # Return after function call
            result = fn(*args, **kwargs)

            # Pop from tha stack after returning
            self.stack.pop()

            # If show_return_value flag is set, display the result
            if self.show_return_value:
                # If shape is set to record
                # Then separate function label and return value by a row
                if "record" in self.node_properties_kwargs.values():
                    function_label = "{" + \
                        function_label + f"|{result} }}"
                else:
                    function_label += f"\n => {result}"

            n_num = self.string2int(function_signature)

            function_label += f"\nDeleted F: {f_deleted_rows[n_num-1]}\nDeleted G: {g_deleted_rows[n_num-1]}"
            if n_num in spliting_dict:
              function_label += f"\nFrequent Variable: {spliting_dict[n_num]}"
            child_node = pydot.Node(name=function_signature,
                                    label=function_label,
                                    **self.node_properties_kwargs)
            self.graph.add_node(child_node)
            # If the function is called by another function
            if caller_func_name not in ['<module>', 'main']:
                # n_num = self.string2int(caller_func_signature)
                # print(n_num)
                # if n_num > len(f_deleted_rows):
                #   k = 2
                # else:
                #   k = 1

                caller_func_label += f"\nDeleted F: {f_deleted_rows[n_num-1]}\nDeleted G: {g_deleted_rows[n_num-1]}"
                if n_num in spliting_dict:
                  caller_func_label += f"\nFrequent Variable: {spliting_dict[n_num]}"
                parent_node = pydot.Node(name=caller_func_signature,
                                         label=caller_func_label,
                                         **self.node_properties_kwargs)
                self.graph.add_node(parent_node)
                edge = pydot.Edge(parent_node, child_node)
                self.graph.add_edge(edge)

            return result
        return wrapper

    @classmethod
    def init_graph(cls):
        # Total number of nodes
        cls.node_count = 0
        cls.graph = pydot.Dot(graph_type="digraph", bgcolor="#fff3af")
        # To track function call numbers
        cls.stack = []
        cls.edges = []
        cls.nodes = []

In [ ]:
#from visualiser.visualiser import Visualiser as vs
# Decorator accepts optional arguments: ignore_args , show_argument_name, show_return_value and node_properties_kwargs

#import pdb
#import pdb; pdb.set_trace() # function to check each step
#spiltting,
#F0=rows that contains x, change the position of x to 0
#F1=rest of the rows
def split(f,g):
    # split function return F0, F1 and g0, g1
    f_sumcol=f.sum(axis=0)
    g_sumcol =g.sum(axis=0)
    ####it should be index= np.argmax(max(f_sumcol, g_sumcol))
    index=np.argmax(f_sumcol+g_sumcol) #find the most freq var(which has the max total column sum in f and g )
    print(f'splitting variable is x{index+1}')
    spliting_dict[x] = f"x{index+1}"
    #In case of multiple occurrences of the maximum values, argmax() makes sure the indices corresponding to the first occurrence are returned.
    # it is proved that the most frequent var has the frequency >=1/log2(|f|+|g|) #Thomas paper theorem 1.3.3
    f0=f[f[:,index]==1]  ## select rows where spliting var column is ==1
    f0[:,index]=0 ##change the  position of splitting var to 0
    f1=f[f[:,index]==0]
    #print(f0,f1)
    g0=g[g[:,index]==1]  ## select rows where spliting var column is ==1
    g0[:,index]=0 ##change the  position of splitting var to 0
    g1=g[g[:,index]==0]
    #return f0,f1,g0,g1
    return f0,f1,g0,g1

def check_base(f,g):
    #|F|=1, |G|=0 or |F|=0, |G|=1
    if np.shape(f)[0]==1 and np.shape(g)[0]==0:
        if f.sum(axis=1)==0:
            #print("passed 1/0")
            return True
        else:
            print("check_base0/1")
            return False
    elif np.shape(g)[0]==1 and np.shape(f)[0]==0:
        if g.sum(axis=1)==0:
            #print("passed 0/1")
            return True
        else:
            print("check_base1/0")
            return False
    if (np.shape(f)[0]*np.shape(g)[0])==0:
        return True
    # |f|=1 and |g|=1: they have and only have 1 same var in the same position, then they are dual to each other
    elif np.shape(f)[0]==1 and np.shape(g)[0] == 1:
            if f.sum(axis=1)==1 and g.sum(axis=1)==1:
                if np.array_equal(f, g):
                    #print("passed 1/1")
                    return True
                else:
                    print("check_base1/1 sum=1, but not equal")
                    return False
            else:
                print("check_base1/1")
                return False

    elif np.shape(f)[0]==1 and np.shape(g)[0]>0:
        # if |f|=1 and |g|=k, easy way to check_dual: sum all columns in g and see if == f
        if np.array_equal(f.sum(axis=0), g.sum(axis=0)):
            for i in range(len(f)):
                zero_index = np.where(f[i] == 1)[0]
            if (np.shape(f)[1]!=np.shape(g)[0]) or (np.shape(f)[1]!=np.shape(g)[1]):
                return False
            for i in zero_index:
                if g[i][i]!=1:
                    print("check_base k/1")
                    return False
            return True
        else:
            print("check_base 1/k")
            return False

    elif np.shape(g)[0]==1 and np.shape(f)[0]>0:
        # if |g|=1 and |f|=k, easy way to check_dual: sum all columns in f and see if == g
        if np.array_equal(g.sum(axis=0), f.sum(axis=0)):
            #record the 1 column index in f
            for i in range(len(g)):
                zero_index = np.where(g[i] == 1)[0]
            if (np.shape(g)[1]!=np.shape(f)[0]) or (np.shape(g)[1]!=np.shape(f)[1]):
                return False
            for i in zero_index:
                if f[i][i]!=1:
                    print("check_base k/1")
                    return False
            return True
        else:
            print("check_base k/1")
            return False

# e.g.: error_path = [path_0, path_1] = [['L', 'R'], ['L','L','L']]
# Add decorator
# show_argument_name=False, show_return_value=False, node_properties_kwargs={"shape":"record", "color":"#f57542", "style":"filled", "fillcolor":"grey"}
@Visualiser(ignore_args=["path"], show_return_value=False, node_properties_kwargs={"shape":"record", "color":"#f57542", "style":"filled", "fillcolor":"grey"})
def fk(f, g, path):
    f, g=remove_duplicate(f,g)
    f, d_f = remove_superset(f)
    g, d_g = remove_superset(g)
    f_remover.append(f)
    g_remover.append(g)
    f_deleted_rows.append(d_f)
    g_deleted_rows.append(d_g)
    if check_base(f, g)==True:
        return True
    if check_base(f, g)==False:
        return False
    elif check_pre(f, g)==False:
        return False
    else:
        print(f"path: {path}")
        print(f"Deleted f = {d_f}\nDeleted g = {d_g}\n")
        f0,f1,g0,g1 = split(f, g)
        #import pdb; pdb.set_trace() # function to check each step
        a0=f1
        a1=np.concatenate((g0, g1),axis=0)
        b0=g1
        b1=np.concatenate((f0,f1), axis=0)
        path.append("L")
        f0_dict[x] = f0
        f1_dict[x] = f1
        g0_dict[x] = g0
        g1_dict[x] = g1
        print(f"\n a0 = {a0}\n\n a1 = {a1}\n\n b0 = {b0}\n\n b1 = {b1}\n")
        # a0_list.append(a0)
        # a1_list.append(a1)
        # b0_list.append(b0)
        # b1_list.append(b1)
        # a_len = len(a0_list)
        fk(f = a0, g = a1, path = path)
        del path[len(path) - 1:]
        #path = path[0:-1]
        path.append("R")
        # b_len = len(b0_list)
        fk(f = b0, g = b1, path = path)
        del path[len(path) - 1:]
        return True

In [ ]:
# Test Cases Compiled.

#Poor cases of the FK-A compiled from the RL Paper by Tamon and Parsa.

# For hypergraphs on 5 variables (n=5), and frequency of MFV (Most Frequent Variable) 1/2,
# np_f= np.array([[0,1,1,0,1],
#                 [1,0,0,1,0]])

# np_g= np.array([[0,1,0,1,0],
#                  [1,1,0,0,0],
#                  [0,0,1,1,0],
#                  [1,0,0,0,1],
#                  [0,0,0,1,1],
#                  [1,0,1,0,0]])

# np_f =np.array( [[0,1,1,0,1],
#        [0,0,0,1,1],
#        [1,1,0,0,0],
#        [1,0,1,1,0]])

# np_g= np.array([[1,0,1,1,0],
#       [0,1,0,1,0],
#       [0,1,1,0,1],
#       [1,0,0,0,1]])


# n=6, Freq= 1/2

# np_f= np.array([[0,1,0,1,0,1],
#     [0,1,0,0,1,0],
#     [1,0,0,1,0,0],
#     [1,0,1,0,1,0]])

# np_g= np.array([[1,0,0,0,1,1],
#     [0,0,0,1,1,0],
#     [1,1,0,0,0,0],
#     [0,1,1,1,0,0]])

# np_f= np.array([[0,0,1,0,1,0],
#       [1,1,0,1,0,1]])

# np_g=np.array([[0,0,1,1,0,0],
#       [1,0,0,0,1,0],
#       [0,1,1,0,0,0],
#       [0,0,0,1,1,0],
#       [0,0,0,0,1,1],
#       [0,0,1,0,0,1],
#       [0,1,0,0,1,0],
#       [1,0,1,0,0,0]])


# np_f=np.array([[0,1,0,0,1,0],
#       [1,0,0,1,0,0],
#       [0,0,1,0,0,1]])

# np_g=np.array([[1,0,1,0,1,0],
#       [1,1,0,0,0,1],
#       [1,1,1,0,0,0],
#       [1,0,0,0,1,1],
#       [0,0,1,1,1,0],
#       [0,1,1,1,0,0],
#       [0,0,0,1,1,1],
#       [0,1,0,1,0,1]])

# np_f= np.array([[0,1,1,0,0,0],
#       [0,1,0,0,1,0],
#       [1,0,0,1,0,0],
#       [0,0,0,1,0,1]])

# np_g= np.array([[1,1,0,0,0,1],
#       [0,1,0,1,0,0],
#       [0,0,1,1,1,0],
#       [1,0,1,0,1,1]])

# n=7, freq = 5/11 (optimal freq = 3/7)

# np_f= np.array([[0,0,1,0,0,1,1],
#     [0,0,1,0,1,1,0],
#     [1,1,1,0,0,0,0],
#     [0,1,0,0,1,0,1],
#     [0,0,0,1,0,1,1],
#     [1,0,0,1,1,0,0],
#     [1,1,0,1,0,0,0]])

# np_g= np.array([[1,1,0,0,0,1,0],
#     [1,0,0,0,1,0,1],
#     [0,0,1,1,0,0,1],
#     [0,1,0,0,1,0,1],
#     [1,0,0,0,1,1,0],
#     [1,0,1,0,0,0,1],
#     [0,0,1,1,1,0,0],
#     [0,1,1,1,0,0,0],
#     [0,1,0,1,0,1,0],
#     [1,0,0,0,0,1,1],
#     [0,1,0,0,1,1,0]])

# Fano Plane (freq value 3/7)
np_f= np.array([[1,1,0,1,0,0,0],
                [0,1,1,0,1,0,0],
                [0,0,1,1,0,1,0],
                [0,0,0,1,1,0,1],
                [1,0,0,0,1,1,0],
                [0,1,0,0,0,1,1],
                [1,0,1,0,0,0,1]])

np_g= np.array([[0,0,0,1,1,0,1],
                [1,0,0,0,1,1,0],
                [1,1,0,1,0,0,0],
                [0,1,1,0,1,0,0],
                [0,0,1,1,0,1,0],
                [0,1,0,0,0,1,1],
                [1,0,1,0,0,0,1]])



# n=8, freq= 2/5 (lower than f2,g2)

# np_f= np.array([[0,1,0,0,0,1,1,1],
#     [0,0,1,0,0,0,1,1],
#     [1,0,0,1,1,0,0,0],
#     [0,1,0,0,1,1,0,0],
#     [1,0,1,1,0,0,0,0]])

# np_g= np.array([[0,0,1,1,0,1,0,0],
#     [0,1,1,0,1,0,0,0],
#     [1,1,0,0,0,0,0,1],
#     [0,0,1,0,1,0,0,1],
#     [0,0,0,1,1,0,1,0],
#     [0,1,0,1,0,0,1,0],
#     [1,0,0,0,1,0,1,0],
#     [1,0,1,0,0,1,0,0],
#     [0,0,0,1,1,0,0,1],
#     [1,1,0,0,0,0,1,0],
#     [0,0,1,0,1,0,1,0],
#     [1,1,1,0,0,0,0,0],
#     [1,0,0,0,0,1,1,0],
#     [1,0,0,0,0,1,0,1],
#     [0,0,0,1,0,1,0,1],
#     [0,0,1,0,1,1,0,0],
#     [0,1,0,1,0,0,0,1],
#     [1,0,0,0,1,0,0,1],
#     [0,1,1,1,0,0,0,0],
#     [0,0,0,1,0,1,1,0]])


# Double-Star
# np_f= np.array([[1,1,0,0,0,0,0,0],
#     [1,0,1,0,0,0,0,0],
#     [1,0,0,1,0,0,0,0],
#     [0,0,0,0,1,1,0,0],
#     [0,0,0,0,1,0,1,0],
#     [0,0,0,0,1,0,0,1]])

# np_g= np.array([[1,0,0,0,1,0,0,0],
#     [1,0,0,0,0,1,1,1],
#     [0,1,1,1,1,0,0,0],
#     [0,1,1,1,0,1,1,1]])


# f2, g2 (Gurvich-Khachiyan Family of Hypergraphs)
# np_f= np.array([[1,0,1,0,0,0,0,0],
#                 [1,0,0,1,0,0,0,0],
#                 [0,1,1,0,0,0,0,0],
#                 [0,1,0,1,0,0,0,0],
#                 [0,0,0,0,1,0,1,0],
#                 [0,0,0,0,1,0,0,1],
#                 [0,0,0,0,0,1,1,0],
#                 [0,0,0,0,0,1,0,1]])

# np_g= np.array([[1,1,0,0,0,0,1,1],
#                 [0,0,1,1,1,1,0,0],
#                 [1,1,0,0,1,1,0,0],
#                 [0,0,1,1,0,0,1,1]])

# np_f= np.array([[1,0,0,0,0,0,0],
#                 [0,1,0,0,0,0,0],
#                 [0,0,1,0,0,1,0],
#                 [0,0,0,1,0,0,0],
#                 [1,0,0,0,1,0,0],
#                 [0,1,0,0,0,1,0],
#                 [1,0,0,0,0,0,1]])
# np_g= np.array([[1,1,1,1,1,0,0]])
#https:\\\\colab.research.google.com\\b00501f2-69fb-4dfd-83b3-b8cebb787ce6

In [ ]:
#test fk
path=[]
res=fk(f = np_f, g = np_g, path = path)
Visualiser.make_animation("fk_demo.gif",delay=1)
if res==True:
    print("\nThey are dual to each other")
else:
    print("\nThey are not dual")

After Running the test fk function, all the f0, g0, f1, g1 values in each node(if created) are printed below the code with the node numbers which can be visualized in the fk_demo.png

In [ ]:
def display(node_num):
  print(f"(F = {f_remover[node_num-1]},\nG = {g_remover[node_num-1]})\n")
  print(f"(F_deleted = {f_deleted_rows[node_num-1]},\nG_deleted = {g_deleted_rows[node_num-1]})\n")
  if node_num in f0_dict:
    print(f"(F0 = {f0_dict[node_num]}\nF1 = {f1_dict[node_num]},\nG0 = {g0_dict[node_num]}\nG1 = {g1_dict[node_num]})\n")
  else:
    print(f"It has no F0, G0, F1, G1 value")
  if node_num in spliting_dict:
    print(f"(Spliting Variable = {spliting_dict[node_num]})\n")
  else:
    print(f"It has no spliting value")

This Function is implemented to display Filtered out by removing supderset Function, Deleted rows and decomposite rows at a specific node number. The node number can be seen from the generated fk_demo.png file

Small Functionality Added

In [ ]:
display(node_num=3)